# Prolonged Length of Stay Prediction

This notebook showcases length of stay prediction on the [Synthea](https://github.com/synthetichealth/synthea) dataset using CyclOps. The task is formulated as a binary classification task, where we predict the probability that a patient will stay 7 days or longer.

To generate the synthetic patient data:

1. Generate synthea data using their releases. We used [v3.0.0](https://github.com/synthetichealth/synthea/releases/tag/v3.0.0).
2. Follow instructions provided in [ETL-Synthea](https://github.com/OHDSI/ETL-Synthea) to load the CSV data into a postgres database.

## Import Libraries

In [ ]:
"""Prolonged Length of Stay Prediction."""

import copy
import os
import shutil
from datetime import date
from pathlib import Path

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datasets import Dataset
from datasets.features import ClassLabel
from dateutil.relativedelta import relativedelta
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

import cyclops.query.ops as qo
from cyclops.data.slicer import SliceSpec
from cyclops.evaluate.fairness import FairnessConfig  # noqa: E402
from cyclops.evaluate.metrics import MetricCollection, create_metric
from cyclops.models.catalog import create_model
from cyclops.process.feature.feature import TabularFeatures
from cyclops.query import DatasetQuerier
from cyclops.report import ModelCardReport
from cyclops.report.plot.classification import ClassificationPlotter
from cyclops.report.utils import flatten_results_dict, get_metrics_trends
from cyclops.tasks.mortality_prediction import MortalityPredictionTask
from cyclops.utils.file import join

CyclOps offers a package for documentation of the model through a model card. The `ModelCardReport` class is used to populate and generate the model card as an HTML file. The model card has the following sections:
- Model Details: This section contains descriptive metadata about the model such as the owners, version, license, etc.
- Model Parameters: This section contains the technical details of the model such as the model architecture, training parameters, etc.
- Considerations: This section contains descriptions of the considerations involved in developing and using the model such as the intended use, limitations, etc.
- Quantitative Analysis: This section contains the performance metrics of the model for different sets of the data and subpopulations.
- Explainaibility Analysis: This section contains the explainability metrics of the model.
- Fairness Analysis: This section contains the fairness metrics of the model.

We will use this to document the model development process as we go along and generate the model card at the end.

`The model card tool is a work in progress and is subject to change.`

In [ ]:
report = ModelCardReport()

## Constants

In [ ]:
NAN_THRESHOLD = 0.25
NUM_DAYS = 7
TRAIN_SIZE = 0.8
RANDOM_SEED = 85

## Data Querying

### Compute length of stay (labels)

1. Get encounters, compute length of stay.
2. Filter out encounters less than 2 days and greater than 20 days.

In [ ]:
querier = DatasetQuerier(
    dbms="postgresql",
    port=5432,
    host="localhost",
    database="synthea_demo",
    user="postgres",
    password="pwd",
)


def get_encounters():
    """Get encounters data."""
    patients = querier.native.patients(
        ops=qo.Sequential([qo.Keep(["id", "birthdate", "gender", "race", "ethnicity"])]),
    )
    ops = qo.Sequential(
        [
            qo.Join(patients.query, on=("patient", "id"), isouter=True),
            qo.ExtractTimestampComponent("start", "year", "start_year"),
            qo.ExtractTimestampComponent("birthdate", "year", "birthdate_year"),
            qo.AddColumn(
                "start_year", "birthdate_year", new_col_labels="age", negative=True,
            ),
            qo.AddColumn("stop", "start", new_col_labels="los", negative=True),
            qo.ConditionGreaterThan("los", 1),
            qo.ConditionLessThan("los", 21),
            qo.Keep(
                [
                    "id",
                    "los",
                    "age",
                    "gender",
                ],
            ),
        ],
    )
    return querier.native.encounters(ops=ops)



def get_observations(cohort):
    """Get observations data."""
    ops = qo.Sequential(
        [
            qo.Join(cohort.query, on=("encounter", "id")),
            qo.ConditionIn(
                "category",
                [
                    "laboratory",
                    "vital-signs",
                ],
            ),
            qo.ConditionEquals("type", "numeric"),
        ],
    )
    observations = querier.native.observations(ops=ops).run()
    ops = qo.Sequential(
        [
            qo.Join(cohort.query, on=("encounter", "id")),
            qo.GroupByAggregate(
                "encounter",
                {"description": ("count", "n_obs")},
            ),
        ],
    )
    observations_count = querier.native.observations(ops=ops).run()
    observations_stats = observations.pivot_table(
        index="encounter", columns="description", values="value", aggfunc="max",
    ).add_prefix("obs_")

    return [observations_count, observations_stats]


def get_medications(cohort):
    """Get medications data."""
    ops = qo.Sequential(
        [
            qo.Join(cohort.query, on=("encounter", "id")),
            qo.GroupByAggregate(
                "encounter",
                {"description": ("count", "n_meds")},
            ),
        ],
    )
    return querier.native.medications(ops=ops).run()



def get_procedures(cohort):
    """Get procedures data."""
    ops = qo.Sequential(
        [
            qo.Join(cohort.query, on=("encounter", "id")),
            qo.GroupByAggregate(
                "encounter",
                {"description": ("count", "n_procedures")},
            ),
        ],
    )
    return querier.native.procedures(ops=ops).run()



def run_query():
    """Run query pipeline."""
    cohort_query = get_encounters()
    to_merge = []
    observations = get_observations(cohort_query)
    to_merge.extend(observations)
    medications = get_medications(cohort_query)
    to_merge.append(medications)
    procedures = get_procedures(cohort_query)
    to_merge.append(procedures)
    cohort = cohort_query.run()
    for to_merge_df in to_merge:
        cohort = cohort.merge(
            to_merge_df, left_on="id", right_on="encounter", how="left",
        )
    to_drop = list(cohort.columns[cohort.columns.str.startswith("encounter")])
    return cohort.drop(columns=to_drop)



cohort = run_query()

## Data Inspection and Preprocessing

### Drop NaNs based on the `NAN_THRESHOLD`

In [ ]:
null_counts = cohort.isnull().sum()[cohort.isnull().sum() > 0]
fig = go.Figure(data=[go.Bar(x=null_counts.index, y=null_counts.values)])

fig.update_layout(
    title="Number of Null Values per Column",
    xaxis_title="Columns",
    yaxis_title="Number of Null Values",
    height=600,
)

fig.show()

**Add the figure to the report**

We can use the log_plotly_figure method to add the figure to a section of the report. One can specify whether the figure should be interactive or not by setting the `interactive` parameter to `True` or `False` respectively. The default value is `True`. This
also affects the final size of the report. If the figure is interactive, the size of the report will be larger than if the figure is not interactive. 

In [ ]:
report.log_plotly_figure(
    fig=fig,
    caption="Number of Null Values per Column",
    section_name="datasets",
    interactive=True,
)

In [ ]:
thresh_nan = int(NAN_THRESHOLD * len(cohort))
cohort = cohort.dropna(axis=1, thresh=thresh_nan)

### Length of stay distribution

In [ ]:
length_of_stay = cohort["los"]
length_of_stay_counts = list(length_of_stay.value_counts().values)
length_of_stay_keys = list(length_of_stay.value_counts().keys())
cohort["outcome"] = cohort["los"] < NUM_DAYS

fig = go.Figure(data=[go.Bar(x=length_of_stay_keys, y=length_of_stay_counts)])
fig.update_layout(
    title="Length of stay",
    xaxis_title="Days",
    yaxis_title="Number of encounters",
    height=600,
)
fig.show()

### Outcome distribution

In [ ]:
cohort["outcome"] = cohort["outcome"].astype("int")
fig = px.pie(cohort, names="outcome")
fig.update_traces(textinfo="percent+label")
fig.update_layout(title_text="Outcome Distribution")
fig.update_traces(
    hovertemplate="Outcome: %{label}<br>Count: \
    %{value}<br>Percent: %{percent}",
)
fig.show()

**Add the figure to the report**

In [ ]:
report.log_plotly_figure(
    fig=fig,
    caption="Outcome Distribution",
    section_name="datasets",
)

In [ ]:
class_counts = cohort["outcome"].value_counts()
class_ratio = class_counts[0] / class_counts[1]
print(class_ratio)

**Add the figure to the report**

In [ ]:
report.log_plotly_figure(
    fig=fig,
    caption="Length of stay distribution",
    section_name="datasets",
)

### Gender distribution

In [ ]:
fig = px.pie(cohort, names="gender")

fig.update_layout(
    title="Gender Distribution",
)

fig.show()

**Add the figure to the report**

In [ ]:
report.log_plotly_figure(
    fig=fig,
    caption="Gender Distribution",
    section_name="datasets",
)

###  Age distribution

In [ ]:
fig = px.histogram(cohort, x="age")
fig.update_layout(
    title="Age Distribution",
    xaxis_title="Age",
    yaxis_title="Count",
    bargap=0.2,
)

fig.show()

**Add the figure to the report**

In [ ]:
report.log_plotly_figure(
    fig=fig,
    caption="Age Distribution",
    section_name="datasets",
)

### Identifying feature types

Cyclops `TabularFeatures` class helps to identify feature types, an essential step before preprocessing the data. Understanding feature types (numerical/categorical/binary) allows us to apply appropriate preprocessing steps for each type.

In [ ]:
features_list = [
    "age",
    "gender",
    "n_obs",
    "n_meds",
    "n_procedures",
    "obs_Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma",
    "obs_Albumin [Mass/volume] in Serum or Plasma",
    "obs_Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma",
    "obs_Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma",
    "obs_Bilirubin.total [Mass/volume] in Serum or Plasma",
    "obs_Body Weight",
    "obs_Calcium [Mass/volume] in Serum or Plasma",
    "obs_Carbon dioxide  total [Moles/volume] in Serum or Plasma",
    "obs_Chloride [Moles/volume] in Serum or Plasma",
    "obs_Creatinine [Mass/volume] in Serum or Plasma",
    "obs_Diastolic Blood Pressure",
    "obs_Erythrocyte distribution width [Ratio] by Automated count",
    "obs_Erythrocytes [#/volume] in Blood by Automated count",
    "obs_Ferritin [Mass/volume] in Serum or Plasma",
    "obs_Glomerular filtration rate/1.73 sq M.predicted",
    "obs_Glucose [Mass/volume] in Serum or Plasma",
    "obs_Hematocrit [Volume Fraction] of Blood by Automated count",
    "obs_Hemoglobin [Mass/volume] in Blood",
    "obs_Leukocytes [#/volume] in Blood by Automated count",
    "obs_MCH [Entitic mass] by Automated count",
    "obs_MCHC [Mass/volume] by Automated count",
    "obs_MCV [Entitic volume] by Automated count",
    "obs_Oxygen saturation in Arterial blood",
    "obs_Platelets [#/volume] in Blood by Automated count",
    "obs_Potassium [Moles/volume] in Serum or Plasma",
    "obs_Protein [Mass/volume] in Serum or Plasma",
    "obs_Sodium [Moles/volume] in Serum or Plasma",
    "obs_Systolic Blood Pressure",
    "obs_Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method",  # noqa: E501
    "obs_Urea nitrogen [Mass/volume] in Serum or Plasma",
]
features_list = sorted(features_list)
tab_features = TabularFeatures(
    data=cohort.reset_index(),
    features=features_list,
    by="id",
    targets="outcome",
)
print(tab_features.types)

### Creating data preprocessors

We create a data preprocessor using sklearn's ColumnTransformer. This helps in applying different preprocessing steps to different columns in the dataframe. For instance, binary features might be processed differently from numeric features.

In [ ]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())],
)

binary_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent"))],
)

In [ ]:
numeric_features = sorted((tab_features.features_by_type("numeric")))
numeric_indices = [
    cohort[features_list].columns.get_loc(column) for column in numeric_features
]
print(numeric_features)

In [ ]:
binary_features = sorted(tab_features.features_by_type("binary"))
binary_features.remove("outcome")
binary_indices = [
    cohort[features_list].columns.get_loc(column) for column in binary_features
]
print(binary_features)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_indices),
        ("onehot", OneHotEncoder(handle_unknown="ignore"), binary_indices),
    ],
    remainder="passthrough",
)

## Creating Hugging Face Dataset

We convert our processed Pandas dataframe into a Hugging Face dataset, a powerful and easy-to-use data format which is also compatible with CyclOps models and evaluator modules. The dataset is then split to train and test sets.

In [ ]:
cohort = cohort.drop(columns=["id", "los"])
dataset = Dataset.from_pandas(cohort)
dataset.cleanup_cache_files()

In [ ]:
dataset = dataset.cast_column("outcome", ClassLabel(num_classes=2))
dataset = dataset.train_test_split(
    train_size=TRAIN_SIZE, stratify_by_column="outcome", seed=RANDOM_SEED,
)

## Model Creation

CyclOps model registry allows for straightforward creation and selection of models. This registry maintains a list of pre-configured models, which can be instantiated with a single line of code. Here we use a SGD classifier to fit a logisitic regression model. The model configurations can be passed to `create_model` based on the sllearn parameters for SGDClassifer.

In [ ]:
model_name = "xgb_classifier"
model = create_model(model_name, random_state=123)

## Task Creation

We use Cyclops tasks to define our model's task (in this case, MortalityPrediction), train the model, make predictions, and evaluate performance. Cyclops task classes encapsulate the entire ML pipeline into a single, cohesive structure, making the process smooth and easy to manage.

In [ ]:
los_task = MortalityPredictionTask(
    {model_name: model}, task_features=features_list, task_target="outcome",
)

In [ ]:
los_task.list_models()

## Training

If `best_model_params` is passed to the `train` method, the best model will be selected after the hyperparameter search. The parameters in `best_model_params` indicate the values to create the parameters grid.

Note that the data preprocessor needs to be passed to the tasks methods if the Hugging Face dataset is not already preprocessed. 

In [ ]:
best_model_params = {
    "n_estimators": [100, 250, 500],
    "learning_rate": [0.1, 0.01],
    "max_depth": [2, 5],
    "reg_lambda": [0, 1, 10],
    "colsample_bytree": [0.7, 0.8, 1],
    "gamma": [0, 1, 2, 10],
    "method": "random",
}
los_task.train(
    dataset["train"],
    model_name=model_name,
    transforms=preprocessor,
    best_model_params=best_model_params,
)

In [ ]:
model_params = los_task.list_models_params()[model_name]
print(model_params)

**Log the model parameters to the report.**

We can add model parameters to the model card using the `log_model_parameters` method.

In [ ]:
report.log_model_parameters(params=model_params)

## Prediction

The prediction output can be either the whole Hugging Face dataset with the prediction columns added to it or the single column containing the predicted values.

In [ ]:
y_pred = los_task.predict(
    dataset["test"],
    model_name=model_name,
    transforms=preprocessor,
    proba=False,
    only_predictions=True,
)
print(len(y_pred))

## Evaluation

Evaluation is done using various evaluation metrics that provide different perspectives on the model's predictive abilities i.e. standard performance metrics and fairness metrics.

The standard performance metrics can be created using the `MetricCollection` object.

In [ ]:
metric_names = [
    "accuracy",
    "precision",
    "recall",
    "f1_score",
    "auroc",
    "roc_curve",
    "precision_recall_curve",
]
metrics = [create_metric(metric_name, task="binary") for metric_name in metric_names]
metric_collection = MetricCollection(metrics)

In addition to overall metrics, it might be interesting to see how the model performs on certain subpopulations. We can define these subpopulations using `SliceSpec` objects. 

In [ ]:
spec_list = [
    {
        "age": {
            "min_value": 20,
            "max_value": 50,
            "min_inclusive": True,
            "max_inclusive": False,
        },
    },
    {
        "age": {
            "min_value": 50,
            "max_value": 80,
            "min_inclusive": True,
            "max_inclusive": False,
        },
    },
    {"gender": {"value": "M"}},
    {"gender": {"value": "F"}},
]
slice_spec = SliceSpec(spec_list)

A `MetricCollection` can also be defined for the fairness metrics.

In [ ]:
specificity = create_metric(
    metric_name="specificity",
    task="binary",
)
sensitivity = create_metric(
    metric_name="sensitivity",
    task="binary",
)

fpr = 1 - specificity
fnr = 1 - sensitivity

ber = (fpr + fnr) / 2

fairness_metric_collection = MetricCollection(
    {
        "Sensitivity": sensitivity,
        "Specificity": specificity,
        "BER": ber,
    },
)

The FairnessConfig helps in setting up and evaluating the fairness of the model predictions.

In [ ]:
fairness_config = FairnessConfig(
    metrics=fairness_metric_collection,
    dataset=None,  # dataset is passed from the evaluator
    target_columns=None,  # target columns are passed from the evaluator
    groups=["gender", "age"],
    group_bins={"age": [20, 40]},
    group_base_values={"age": 40, "gender": "M"},
    thresholds=[0.5],
)

The evaluate methods outputs the evaluation results and the Hugging Face dataset with the predictions added to it.

In [ ]:
results, dataset_with_preds = los_task.evaluate(
    dataset["test"],
    metric_collection,
    model_names=model_name,
    transforms=preprocessor,
    prediction_column_prefix="preds",
    slice_spec=slice_spec,
    batch_size=64,
    fairness_config=fairness_config,
    override_fairness_metrics=False,
)

**Log the performance metrics to the report.**

We can add a performance metric to the model card using the `log_performance_metric` method, which expects a dictionary where the keys are in the following format: `slice_name/metric_name`. For instance, `overall/accuracy`. 

We first need to process the evaluation results to get the metrics in the right format.

In [ ]:
results_flat = flatten_results_dict(
    results=results,
    remove_metrics=["BinaryROCCurve", "BinaryPrecisionRecallCurve"],
    model_name=model_name,
)
print(results_flat)

In [ ]:
for name, metric in results_flat.items():
    split, name = name.split("/")  # noqa: PLW2901
    pass_fail_thresholds = 0.8 if name == "BinaryAUROC" else 0.6
    report.log_quantitative_analysis(
        "performance",
        name=name,
        value=metric,
        metric_slice=split,
        pass_fail_thresholds=pass_fail_thresholds,
        pass_fail_threshold_fns=lambda x, threshold: x >= threshold,
    )

We can also use the `ClassificationPlotter` to plot the performance metrics and the add the figure to the model card using the `log_plotly_figure` method.

In [ ]:
plotter = ClassificationPlotter(task_type="binary", class_names=["0", "1"])
plotter.set_template("plotly_white")

In [ ]:
# extracting the ROC curves and AUROC results for all the slices
roc_curves = {
    slice_name: slice_results["BinaryROCCurve"]
    for slice_name, slice_results in results[model_name].items()
}
aurocs = {
    slice_name: slice_results["BinaryAUROC"]
    for slice_name, slice_results in results[model_name].items()
}
roc_curves.keys()

In [ ]:
# plotting the ROC curves for all the slices
roc_plot = plotter.roc_curve_comparison(roc_curves, aurocs=aurocs)
report.log_plotly_figure(
    fig=roc_plot,
    caption="ROC Curve for Female Patients",
    section_name="quantitative analysis",
)
roc_plot.show()

In [ ]:
# Extracting the overall classification metric values.
overall_performance = {
    metric_name: metric_value
    for metric_name, metric_value in results[model_name]["overall"].items()
    if metric_name not in ["BinaryROCCurve", "BinaryPrecisionRecallCurve"]
}

In [ ]:
# Plotting the overall classification metric values.
overall_performance_plot = plotter.metrics_value(
    overall_performance, title="Overall Performance",
)
report.log_plotly_figure(
    fig=overall_performance_plot,
    caption="Overall Performance",
    section_name="quantitative analysis",
)
overall_performance_plot.show()

In [ ]:
# Extracting the metric values for all the slices.
slice_metrics = {
    slice_name: {
        metric_name: metric_value
        for metric_name, metric_value in slice_results.items()
        if metric_name not in ["BinaryROCCurve", "BinaryPrecisionRecallCurve"]
    }
    for slice_name, slice_results in results[model_name].items()
}

In [ ]:
# Plotting the metric values for all the slices.
slice_metrics_plot = plotter.metrics_comparison_bar(slice_metrics)
report.log_plotly_figure(
    fig=slice_metrics_plot,
    caption="Slice Metric Comparison",
    section_name="quantitative analysis",
)
slice_metrics_plot.show()

In [ ]:
# Reformating the fairness metrics
fairness_results = copy.deepcopy(results["fairness"])
fairness_metrics = {}
# remove the group size from the fairness results and add it to the slice name
for slice_name, slice_results in fairness_results.items():
    group_size = slice_results.pop("Group Size")
    fairness_metrics[f"{slice_name} (Size={group_size})"] = slice_results

In [ ]:
# Plotting the fairness metrics
fairness_plot = plotter.metrics_comparison_scatter(
    fairness_metrics, title="Fairness Metrics",
)
report.log_plotly_figure(
    fig=fairness_plot,
    caption="Fairness Metrics",
    section_name="fairness analysis",
)
fairness_plot.show()

### Performance over time

We can monitor the change of performance metrics over time by leveraging historical reports. This involves using the `get_metrics_trends` function to gather prior metrics and merge them with recent results. We can specify which metrics and slices we wish to observe. Once the data is collected, we generate the plot using the `metrics_trends` method from the plotter, which can then be integrated into the report.

Please note, for the purpose of this tutorial, we will create three dummy reports to demonstrate the process of plotting these metric trends.

In [ ]:
# Generating dummy reports
dummy_report_num = 3
dummy_report_dir = join(os.getcwd(), "dummy_reports")

for i in range(dummy_report_num):
    # Create a dummy model card report
    dummy_report = ModelCardReport(output_dir=dummy_report_dir)
    # Add noise to the recent results to simulate the model performance change
    noise = np.random.uniform(-0.05, 0.05)
    dummy_result = {key: max(0, value - noise) for key, value in results_flat.items()}
    for name, metric in dummy_result.items():
        split, name = name.split("/")  # noqa: PLW2901
        pass_fail_thresholds = 0.8 if name == "BinaryAUROC" else 0.6
        dummy_report.log_quantitative_analysis(
            "performance",
            name=name,
            value=metric,
            metric_slice=split,
            pass_fail_thresholds=0.8,
            pass_fail_threshold_fns=lambda x, threshold: x >= threshold,
        )
    # Rename the report folder to the dummy date to simulate the time change
    dummy_report_path = dummy_report.export()
    date_dir = Path(dummy_report_path).parents[1]
    dummy_date = date.today() + relativedelta(months=-(6 * (i + 1)))
    new_dir = f"{dummy_report_dir}/{dummy_date}"
    if os.path.exists(new_dir):
        shutil.rmtree(new_dir)
    os.rename(date_dir, new_dir)

In [ ]:
# Collecting performance metrics from previous reports and current report
trends = get_metrics_trends(
    report_directory=dummy_report_dir,
    flat_results=results_flat,
    slice_names=["overall"],
)
shutil.rmtree(dummy_report_dir)

In [ ]:
# Plotting the performance over time
trends_plot = plotter.metrics_trends(trends)
report.log_plotly_figure(
    fig=trends_plot,
    caption="Performance over time",
    section_name="quantitative analysis",
)
trends_plot.show()

## Report Generation

Before generating the model card, let us document some of the details of the model and some considerations involved in developing and using the model.


Let's start with populating the model details section, which includes the following fields by default:
- description: A high-level description of the model and its usage for a general audience.
- version: The version of the model.
- owners: The individuals or organizations that own the model.
- license: The license under which the model is made available.
- citation: The citation for the model.
- references: Links to resources that are relevant to the model.
- path: The path to where the model is stored.
- regulatory_requirements: The regulatory requirements that are relevant to the model.

We can add additional fields to the model details section by passing a dictionary to the `log_from_dict` method and specifying the section name as `model_details`. You can also use the `log_descriptor` method to add a new field object with a `description` attribute to any section of the model card.

In [ ]:
report.log_from_dict(
    data={
        "description": "The model was trained on the Synthea synthetic dataset \
            to predict prolonged stay in the hospital.",
    },
    section_name="model_details",
)
report.log_version(
    version_str="0.0.1",
    date=str(date.today()),
    description="Initial Release",
)
report.log_owner(name="CyclOps Team", contact="vectorinstitute.github.io/cyclops/")
report.log_license(identifier="Apache-2.0")
report.log_reference(
    link="https://xgboost.readthedocs.io/en/stable/python/python_api.html",  # noqa: E501
)

Next, let's populate the considerations section, which includes the following fields by default:
- users: The intended users of the model.
- use_cases: The use cases for the model. These could be primary, downstream or out-of-scope use cases.
- fairness_assessment: A description of the benefits and harms of the model for different groups as well as the steps taken to mitigate the harms.
- ethical_considerations: The risks associated with using the model and the steps taken to mitigate them. This can be populated using the  `log_risk` method.



In [ ]:
report.log_from_dict(
    data={
        "users": [
            {"description": "Hospitals"},
            {"description": "Clinicians"},
        ],
    },
    section_name="considerations",
)
report.log_user(description="ML Engineers")
report.log_use_case(
    description="Predicting prolonged length of stay",
    kind="primary",
)
report.log_fairness_assessment(
    affected_group="sex, age",
    benefit="Improved health outcomes for patients.",
    harm="Biased predictions for patients in certain groups (e.g. older patients) \
        may lead to worse health outcomes.",
    mitigation_strategy="We will monitor the performance of the model on these groups \
        and retrain the model if the performance drops below a certain threshold.",
)
report.log_risk(
    risk="The model may be used to make decisions that affect the health of patients.",
    mitigation_strategy="The model should be continuously monitored for performance \
        and retrained if the performance drops below a certain threshold.",
)

Once the model card is populated, you can generate the report using the `export` method. The report is generated in the form of an HTML file. A JSON file containing the model card data will also be generated along with the HTML file. By default, the files will be saved in a folder named `cyclops_reports` in the current working directory. You can change the path by passing a `output_dir` argument when instantiating the `ModelCardReport` class.

In [ ]:
report_path = report.export()
shutil.copy(f"{report_path}", ".")

You can view the generated HTML [report](./model_card.html).